### Preprocessing for HDAC6
HDAC6 is a type of *zinc metalloprotein*, and zinc atoms are not supported in the Vina force field. Therefore, docking with HDAC6 requires the use of the special updated force field, AutoDock4Zn.  
This notebook will implement the preparation process for the HDAC6 protein based on the tutorial from the [official documentation](https://autodock-vina.readthedocs.io/en/latest/docking_zinc.html) of AutoDock-Vina.

ADFR suite is required for the following steps.

In [1]:
from vina import Vina
import subprocess
import rdkit.Chem as Chem
from rdkit.Chem import AllChem

script_dir = '../../Dock/autodock_scripts'
lig_prefix = 'HDAC6_8bjk_ligand_A'
rec_prefix = 'HDAC6_8bjk_rec_A'

#### Receptor and ligand preparation

In [2]:
# Convert to pdbqt
command_prep = f'''mk_prepare_receptor.py -i {rec_prefix}.pdb -o {rec_prefix} -p'''
proc = subprocess.Popen(command_prep, shell=True, stdin=subprocess.PIPE, 
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

for line in proc.stdout:
    print(line.strip().decode())


Files written:
HDAC6_8bjk_rec_A.pdbqt <-- static (i.e., rigid) receptor input file


In [3]:
# Add zinc pseudo atom
command_zinc = f'''pythonsh {script_dir}/zinc_pseudo.py -r {rec_prefix}.pdbqt -o {rec_prefix}_tz.pdbqt'''
proc = subprocess.Popen(command_zinc, shell=True, stdin=subprocess.PIPE, 
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

for line in proc.stdout:
    print(line.strip().decode())

Wrote 1 TZ atoms on HDAC6_8bjk_rec_A_tz.pdbqt.


In [4]:
command_ligprep = f'''mk_prepare_ligand.py -i {lig_prefix}.sdf -o {lig_prefix}.pdbqt'''
proc = subprocess.Popen(command_ligprep, shell=True, stdin=subprocess.PIPE, 
                        stdout=subprocess.PIPE, stderr=subprocess.PIPE)

for line in proc.stdout:
    print(line.strip().decode())

Input molecules processed: 1, skipped: 0
PDBQT files written: 1
PDBQT files not written due to error: 0
Input molecules with errors: 0


#### Generating affinity maps

**AD4Zn.dat** must be downloaded in the working directory before running the code below.  
It can be download at [here](https://github.com/ccsb-scripps/AutoDock-Vina/blob/develop/data/AD4Zn.dat).

In [5]:
# Get the centroid of ligand as the box center
def sdf2centroid(sdf_file):
    supp = Chem.SDMolSupplier(sdf_file, sanitize=False)
    lig_xyz = supp[0].GetConformer().GetPositions()
    centroid_x = '%.4f'% lig_xyz[:,0].mean()
    centroid_y = '%.4f'% lig_xyz[:,1].mean()
    centroid_z = '%.4f'% lig_xyz[:,2].mean()
    return centroid_x, centroid_y, centroid_z

centroid_x, centroid_y, centroid_z = sdf2centroid(f'{lig_prefix}.sdf')
centroid_x, centroid_y, centroid_z 

('4.6857', '2.6914', '28.8837')

In [6]:
# Set the grid box
# Here the box size were set as 19.875Å * 19.875Å * 19.875Å
# e.g. (num of grid points - 1) * 0.375Å

gpf4zn = f'''pythonsh {script_dir}/prepare_gpf4zn.py \
                -l {lig_prefix}.pdbqt \
                -r {rec_prefix}_tz.pdbqt \
                -o protein_tz.gpf \
                -p npts=54,54,54\
                -p gridcenter={centroid_x},{centroid_y},{centroid_z}  \
                –p parameter_file=AD4Zn.dat'''

proc = subprocess.Popen(
                gpf4zn, 
                shell=True, 
                stdin=subprocess.PIPE, 
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE)

for line in proc.stdout:
    print(line.strip().decode())

#### Generate the docking grid

In [7]:
command_grid = '''autogrid4 -p protein_tz.gpf -l protein_tz.glg'''
proc = subprocess.Popen(
                command_grid, 
                shell=True, 
                stdin=subprocess.PIPE, 
                stdout=subprocess.PIPE, 
                stderr=subprocess.PIPE)

for line in proc.stdout:
    print(line.strip().decode())

In [8]:
# Move all maps file to ./maps/
!mkdir maps
!mv HDAC6_8bjk_rec_A_tz.* maps